In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import tkinter
%pylab inline
import pandas as pd
from itertools import cycle, islice
from functools import reduce
from scipy.stats import linregress
import scipy.stats as st
import numpy as np

# Source data files

ev_Sales_path = "data/Electric_Car_Sales_by_Model_in_USA.csv"
#ev_path = "data/electric_cars.csv"
ev1_path = "data/ElectricCarData_Clean_Me.csv"
ev2_path = "data/Quickestelectriccars-EVDatabase.csv"
Vehicles_path = "data/Vehicles.csv"


# Read the EV data and the Source data results

ev_Sales = pd.read_csv(ev_Sales_path)
#ev = pd.read_csv(ev_path)
ev1 = pd.read_csv(ev1_path)
ev2 = pd.read_csv(ev2_path)
Vehicles = pd.read_csv(Vehicles_path)

Populating the interactive namespace from numpy and matplotlib


In [82]:
ev_Sales.head()

,Make,Model,Logo,janv-12,Feb 2012,mars-12,Apr 2012,May 2012,juin-12,juil-12,...,mars-19,Apr 2019,May 2019,juin-19,juil-19,Aug 2019,sept-19,oct-19,nov-19,Dec 2019
0,Chevrolet,Volt,https://www.carlogos.org/logo/Chevrolet-logo-2...,603.0,1626.0,3915.0,5377.0,7057.0,8817.0,10666.0,...,146667.0,147072.0,"147,48","147,813","148,063","148,337","148,687","148,757","148,907","149,057"
1,Toyota,Prius PHV,https://www.carlogos.org/logo/Toyota-logo-1989...,0.0,21.0,912.0,2566.0,3652.0,4347.0,5035.0,...,42345.0,42345.0,42345,42345,42345,42345,42345,42345,42345,42345
2,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,122534.0,123485.0,"124,701","125,857","126,795","127,912","128,96","129,847","130,987","132,214"
3,Tesla,Model S,https://www.carlogos.org/logo/Tesla-logo-2003-...,0.0,NaN,NaN,NaN,NaN,12.0,31.0,...,147517.0,148342.0,"149,367","151,117","152,092","153,142","154,242","154,992","156,492","157,992"
4,Ford,C-Max Energi,https://www.carlogos.org/logo/Ford-logo-2003-1...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,42231.0,42231.0,42231,42231,42231,42231,42231,42231,42231,42231


In [73]:
Vehicles.columns

Index(['Automobile ID', 'Fuel ID', 'Fuel Configuration ID', 'Manufacturer ID',
       'Weight Class ID', 'Category ID', 'Model', 'Model Year', 'City Range',
       'Highway Range', 'City Fuel Economy', 'City Fuel Economy Units',
       'Highway Fuel Economy', 'Highway Fuel Economy Units',
       'City Gas Fuel Economy', 'Highway Gas Fuel Economy',
       'Transmission Type', 'Engine Type', 'Engine Size',
       'Engine Cylinder Count', 'Engine Description', 'Battery Type',
       'Battery Voltage Amps', '110v Charge Time', '220v Charge Time',
       'Warranty Info', 'Ordering Info', 'Description', 'Comments',
       'Manufacturer', 'Manufacturer URL', 'Fuel Code', 'Fuel', 'Category',
       'Fuel Configuration Name'],
      dtype='object')

In [76]:
#Merge the DataFrames
merged_df = pd.merge(ev_Sales, Vehicles[['Model','Model Year','City Range','Highway Range','Transmission Type', 'Engine Type', 'Engine Size','Engine Cylinder Count', 'Engine Description', 'Battery Type',
       'Battery Voltage Amps', '110v Charge Time', '220v Charge Time','Manufacturer','Fuel Code', 'Fuel', 'Category']], right_on=['Model'],left_on=['Model'], how='inner')
merged_df.columns

Index(['Make', 'Model', 'Logo', 'janv-12', 'Feb 2012', 'mars-12', 'Apr 2012',
       'May 2012', 'juin-12', 'juil-12',
       ...
       'Engine Cylinder Count', 'Engine Description', 'Battery Type',
       'Battery Voltage Amps', '110v Charge Time', '220v Charge Time',
       'Manufacturer', 'Fuel Code', 'Fuel', 'Category'],
      dtype='object', length=115)

In [80]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Columns: 115 entries, Make to Category
dtypes: float64(89), object(26)
memory usage: 26.3+ KB


In [85]:
merged_df.head()

,Make,Model,Logo,janv-12,Feb 2012,mars-12,Apr 2012,May 2012,juin-12,juil-12,...,Engine Cylinder Count,Engine Description,Battery Type,Battery Voltage Amps,110v Charge Time,220v Charge Time,Manufacturer,Fuel Code,Fuel,Category
0,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,NaN,80 kW electric motor 24 kWh battery,NaN,24 kWh,NaN,NaN,Nissan,ELEC,Electric,Sedan/Wagon
1,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,NaN,"80 kW electric motor, 24 kWh battery",NaN,24 kWh,NaN,NaN,Nissan,ELEC,Electric,Sedan/Wagon
2,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,NaN,80 kW electric motor,NaN,24 kWh,NaN,NaN,Nissan,ELEC,Electric,Sedan/Wagon
3,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,NaN,NaN,24 kWh,NaN,NaN,NaN,Nissan,ELEC,Electric,Sedan/Wagon
4,Nissan,Leaf,https://www.carlogos.org/logo/Nissan-logo-2013...,676.0,1154.0,1733.0,2103.0,2613.0,3148.0,3543.0,...,NaN,NaN,Li-Ion,NaN,NaN,NaN,Nissan,ELEC,Electric,Sedan/Wagon


In [88]:
merged_df.to_csv("testdata.csv", index=False)